In [1]:
import numpy as np

In [32]:
# Inputs
Mufy = 324054
MuS = 8.2e-4
Muq = 29.19
Sigfy = 32405.4
SigS = 8.2e-5
Sigq = 5.838

In [102]:
# Define a function to calculate beta from x*
def calculatebeta(mpfx, mux, sigx, partialgx):
    numerator = np.sum(mux*partialgx)-(mpfx[0]*mpfx[1])
    denominator = np.sqrt(np.sum(sigx*sigx*partialgx*partialgx))
    beta = numerator/denominator
    return beta  

In [103]:
# Define a function to calculate x*' from beta 
def calculatempfxp(sigx, partialgx, beta):
    numerator = sigx*partialgx
    denominator = np.sqrt(np.sum(sigx*sigx*partialgx*partialgx))
    alpha = numerator/denominator
    mpfxp = -beta * alpha
    return mpfxp  

In [104]:
def iteratxbeta(initialmpfxp, mux, sigx, l):
    # Itinialize the discrepancy between the iterated reduced mpfx
    discrep = 10
    mpfxp = initialmpfxp
    mpfx = initialmpfxp * sigx + mux
    # Define a list to store the iterative procedure
    mpfxlist = []
    betalist = []
    # Define a criteria for convergence
    criteria = 1e-8
    while discrep > criteria:
        # Calculate beta
        partialgx = np.array([mpfx[1], mpfx[0], -(l*l/8)], dtype=np.float64)
        beta = calculatebeta(mpfx, mux, sigx, partialgx)
        # Update mpfx
        mpfxpnew = calculatempfxp(sigx, partialgx, beta)
        mpfx = mpfxpnew*sigx + mux
        # Set the Eulerian distance betweeen the mpfxp vector as the discrepancy
        discrep = np.linalg.norm(mpfxpnew-mpfxp)
        # Update mpfxp
        mpfxp = mpfxpnew
        mpfxlist.append(mpfx)
        betalist.append(beta)
    return mpfxlist, betalist 

In [98]:
Mux = np.array([Mufy, MuS, Muq], dtype=np.float64)
Sigx = np.array([Sigfy, SigS, Sigq], dtype=np.float64)
Initialmpfxp = np.array([0, 0, 0]) 
L = 6.1

In [105]:
Mpfxlist, Betalist = iteratxbeta(Initialmpfxp, Mux, Sigx, L)

In [106]:
Mpfxlist

[array([2.71994947e+05, 6.88267563e-04, 3.87739712e+01]),
 array([2.72693458e+05, 6.90035104e-04, 4.04551087e+01]),
 array([2.72636965e+05, 6.89892151e-04, 4.04386120e+01]),
 array([2.72641483e+05, 6.89903585e-04, 4.04399541e+01]),
 array([2.72641122e+05, 6.89902669e-04, 4.04398468e+01]),
 array([2.72641150e+05, 6.89902743e-04, 4.04398554e+01]),
 array([2.72641148e+05, 6.89902737e-04, 4.04398547e+01]),
 array([2.72641148e+05, 6.89902737e-04, 4.04398548e+01]),
 array([2.72641148e+05, 6.89902737e-04, 4.04398548e+01])]

In [107]:
Betalist

[2.8029744073813334,
 2.957613933667967,
 2.9576411772649256,
 2.9576413555321723,
 2.9576413566725286,
 2.9576413566798383,
 2.957641356679883,
 2.9576413566798845,
 2.9576413566798854]

In [108]:
from scipy.stats import norm

In [109]:
# Output the failure probability 
print(1-norm.cdf(2.9576))

0.0015502208227223813


In [113]:
# Use PAC Monte Carlo to calculate the pf
def failurestate(mux, sigx, l):
    # Sample
    x1 = np.random.normal(loc=mux[0], scale=sigx[0], size=1)[0]
    x2 = np.random.normal(loc=mux[1], scale=sigx[1], size=1)[0]
    x3 = np.random.normal(loc=mux[2], scale=sigx[2], size=1)[0]
    # Check if failure occurs
    state = 0
    if x1*x2-(x3*l*l/8) < 0:
        state = 1
    return state 

In [111]:
# Implement the GBAS algorithm
def GBAS(k, mux, sigx, l):
    # Initialize variable
    scount = 0
    qcount = 0
    samplesize = 0
    while scount < k:
        # Repeat MCS
        scount += failurestate(mux, sigx, l)
        qcount += np.random.exponential(scale = 1.0, size = 1)[0]
        samplesize += 1
    r = (k-1)/qcount
    return r, samplesize 

In [138]:
# k parameter for GBAS for Epsilon = 0.05, Delta = 0.05 K = 1550, E=0.02, D=0.01 K=16600
K = 16600
Pf, Samplesize = GBAS(K, Mux, Sigx, L)

In [139]:
Pf

0.001646677673261173

In [143]:
Samplesize

10078792

In [116]:
print(0.12/1.69)

0.07100591715976332


In [132]:
print(1/1.02)

0.9803921568627451


In [ ]:
0.98039

In [144]:
x = np.array([2.72641150e+05, 6.89902743e-04, 4.04398554e+01])
xp = (x - Mux)/Sigx
print(xp)

[-1.58655193 -1.58655191  1.92700504]


In [142]:
print(1.647/0.98)

1.6806122448979592
